In [ ]:
! pip install datasets transformers[torch] seqeval
! pip install accelarate
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_c

ERROR: Could not find a version that satisfies the requirement accelarate (from versions: none)
ERROR: No matching distribution found for accelarate
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
import numpy as np
import evaluate
import json
import csv
from torch.utils.data import Dataset
from datasets import *
from transformers import AutoModelForTokenClassification,AutoModelForSequenceClassification, Trainer, AutoTokenizer, DistilBertForTokenClassification, TrainingArguments, DataCollatorWithPadding
from sklearn.model_selection import train_test_split

Loading, Tansforming and splitting the Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_excel('/content/drive/My Drive/SmishingData/GhanaContextMessages.xlsx')

In [ ]:
data.columns = ['label', 'text']

In [ ]:
# Split the data into training and testing sets
train_df, test_df = train_test_split(data, test_size=0.3, random_state=42, shuffle=True)
print(f"new dataframe shapes: train is {train_df.shape}, eval is {test_df.shape}")

new dataframe shapes: train is (964, 2), eval is (414, 2)


In [ ]:
# Converting the Dataframes to DatasetDict because the Distillbert requires the dataset to be stored in a variable of type DatasetDict.
ds = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test": Dataset.from_pandas(test_df),
    "validation": Dataset.from_pandas(test_df)
})

In [ ]:
ds

In [ ]:
def transform_labels(label):
    label = label['label']
    num = 0
    if label == 'ham': #'Negative'
        num = 0
    elif label == 'smish': #'Positive'
        num = 1
    return {'label': num}


ds = ds.map(transform_labels)

Map:   0%|          | 0/964 [00:00<?, ? examples/s]

Map:   0%|          | 0/414 [00:00<?, ? examples/s]

Map:   0%|          | 0/414 [00:00<?, ? examples/s]

In [ ]:
#Removing unecessary columns
ds['train'] = ds['train'].remove_columns(['__index_level_0__'])
ds['test'] = ds['test'].remove_columns(['__index_level_0__'])
ds['validation'] = ds['validation'].remove_columns(['__index_level_0__'])

In [ ]:
ds

Tokenizing the data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenization(example):  # Putting examples in lowercase as distilbert is uncased
    lowercased_example = list(map(str.lower, example['text']))
    return tokenizer(lowercased_example, truncation=True)

# Tokenizing the messages in batches and removing the original column
tokenized_dataset = ds.map(tokenization, batched=True, remove_columns=['text'])
tokenized_dataset

Map:   0%|          | 0/964 [00:00<?, ? examples/s]

Map:   0%|          | 0/414 [00:00<?, ? examples/s]

Map:   0%|          | 0/414 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 964
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 414
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 414
    })
})

Training

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2, id2label = {0:'ham',1:'smish'}, label2id = {'ham':0,'smish':1})  #model initialization

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer)

In [ ]:
#pip install accelerate -U

In [ ]:
!pip show accelerate

Initialization for calculations of Metrics



In [ ]:
accuracy_metric = evaluate.load('accuracy')
f1_metric = evaluate.load('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    return {**accuracy, **f1}



In [ ]:
training_args = TrainingArguments(
    seed=42,
    output_dir='./results',
    num_train_epochs=3,
    save_strategy='epoch',
    evaluation_strategy='epoch',
    load_best_model_at_end=True,

)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.107996,0.973430,0.969014
2,No log,0.046319,0.990338,0.988950
3,No log,0.017695,0.995169,0.994505


TrainOutput(global_step=363, training_loss=0.06994515411124742, metrics={'train_runtime': 1235.8209, 'train_samples_per_second': 2.34, 'train_steps_per_second': 0.294, 'total_flos': 43108824148560.0, 'train_loss': 0.06994515411124742, 'epoch': 3.0})

Evaluation

In [ ]:
trainer.evaluate()

{'eval_loss': 0.01769460178911686,
 'eval_accuracy': 0.9951690821256038,
 'eval_f1': 0.9945054945054945,
 'eval_runtime': 40.3531,
 'eval_samples_per_second': 10.259,
 'eval_steps_per_second': 1.289,
 'epoch': 3.0}

In [ ]:
results = trainer.evaluate(tokenized_dataset['test'])
results

{'eval_loss': 0.01769460178911686,
 'eval_accuracy': 0.9951690821256038,
 'eval_f1': 0.9945054945054945,
 'eval_runtime': 38.112,
 'eval_samples_per_second': 10.863,
 'eval_steps_per_second': 1.364,
 'epoch': 3.0}

In [ ]:
results_path =  "/content/drive/My Drive/SmishingData/Results/Test_Results.json"
with open(results_path, "w") as file:
    json.dump(results, file)

Saving Model

In [ ]:
output_dir = "/content/drive/My Drive/SmishingData/Model/GhContext_model"
trainer.save_model(output_dir)

Prediction

In [ ]:
#importing model
gh_model = AutoModelForTokenClassification.from_pretrained(output_dir)

In [ ]:
test_results = trainer.predict(tokenized_dataset["test"])

In [ ]:
# Assuming test_results is the output from trainer.predict(test_dataset)
logits = test_results.predictions
label_ids = test_results.label_ids

# Convert logits to probabilities and then to class predictions
probabilities = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()
predicted_classes = np.argmax(probabilities, axis=1)

predicted_labels = ['smish' if pred == 1 else 'ham' for pred in predicted_classes]
actual_labels = ['smish' if label == 1 else 'ham' for label in label_ids]

# Extract text from DataFrame
test_texts = test_df['text'].tolist()
for text, predicted_label, actual_label in zip(test_texts, predicted_labels, actual_labels):
    print(f"Text: {text}\nPredicted Class: {predicted_label}, Actual Class: {actual_label}\n")

Text: Transaction of GHS120.00 to UltraPay finalized. New Balance: GHS 274.26. Transaction Id: 79463353435. No fee charged. Quick payments via the MoMo App. Link: http://mtnghana.app.link/nsBnhItDoob
Predicted Class: ham, Actual Class: ham

Text: Cash Received: GHS 650.00 from NEXT GEN SOLUTIONS. Verify this transaction at no cost at https://secureconfirm.com
Predicted Class: smish, Actual Class: smish

Text: Assembling a team for various remote tasks, daily pay: 3100-5700, start via WhatsApp: http://wa.me/233557332202
Predicted Class: smish, Actual Class: smish

Text: Earn from your smartphone, immediate payouts of 200-220 USD, start today, link: https://wa.me/639163056598
Predicted Class: smish, Actual Class: smish

Text: Remote job opportunities now available, earn GHS250-900 daily from your home, start immediately: http://wa.me/447931662349
Predicted Class: smish, Actual Class: smish

Text: Take your chance! 0544488144, win GHS 200,000! Just type WIN to join. Reply WIN
Predicted Cl

Interface directly with Model

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def predict(text):
    # Prepare the text for the model
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)

    # Get predictions
    with torch.no_grad():
        outputs = model(**inputs)

    #Getting probabilities
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)

    # Extract the confidence and the predicted class
    confidence, predicted_class = torch.max(probabilities, dim=-1)

    # DIsplaying text in redable format
    predicted_label = 'smish' if predicted_class.item() == 1 else 'ham'
    confidence_score = confidence.item() * 100  # Convert to percentage

    return predicted_label, confidence_score

In [ ]:
print("Type 'quit' to exit.")
while True:
    user_input = input("Enter a message to classify: ")
    if user_input.lower() == 'quit':
        break
    predicted_label, confidence_score = predict(user_input)
    print(f"Predicted Class: {predicted_label} with {confidence_score:.2f}% confidence.\n")